# "A/B tests with hashes in BigQuery"

- author: bbernst
- toc: true
- branch: master
- badges: true
- comments: true
- categories: [bigquery, "ab tests"]
- image: images/bigquery-ab-tests/dists.png
- hide: false
- search_exclude: true

# An example problem

Recently, I've needed to generate random but consistent values in a BigQuery data pipeline. This is a pipeline that runs many times per day and constantly has new data feeding in through upstream pipes.

As an example, let's say that we have 10,000 users and need to randomly place them into groups depending on the hour of the work day. Additionally, we only want to do this once; once a user gets placed into a group, they stay in that group. This way, downstream work can rely on how each user is assigned without having to worry about checking for changes. Visually, this looks like:

In [1]:
#hide
import pandas as pd

In [2]:
#hide
hours = range(9, 19)
pd.DataFrame({'hour': hours, 'user_ids': ['~1000 random users']*len(hours)})

,hour,user_ids
0,9,~1000 random users
1,10,~1000 random users
2,11,~1000 random users
3,12,~1000 random users
4,13,~1000 random users
5,14,~1000 random users
6,15,~1000 random users
7,16,~1000 random users
8,17,~1000 random users
9,18,~1000 random users


BigQuery has a number of functions for producing random numbers, like `RAND()`, and we can create the table above with a query:

```sql
WITH src AS (
  SELECT
    user_id,
    CAST(FLOOR(9 + 10 * RAND()) AS INT64) AS hour
  FROM UNNEST(GENERATE_ARRAY(1, 10000)) user_id
)

SELECT 
  hour,
  ARRAY_AGG(user_id) AS user_ids
FROM src
GROUP BY hour
ORDER BY hour
```

However, this query produces *NEW* random groups each time the query is run. There's no built-in way to specify a seed in `RAND()`, so we'll need to learn how to create a `CONSISTENT_RAND(...)` ourselves.

# Solution

This [thorough post](http://blog.richardweiss.org/2016/12/25/hash-splits.html) solves our problem, gives us python and SQL code, but unfortunately not for the BigQuery dialect.

Let's build this up for BigQuery step by step within the `src` CTE:

```sql
  SELECT
    user_id,
    CONCAT(user_id, 'salt') AS key,
    MD5(CONCAT(user_id, 'salt')) AS md5_key,
    TO_HEX(MD5(CONCAT(user_id, 'salt'))) AS hex_md5_key,
    SUBSTR(TO_HEX(MD5(CONCAT(user_id, 'salt'))), 1, 6) AS substr_hex_md5_key,
    CONCAT('0x', SUBSTR(TO_HEX(MD5(CONCAT(user_id, 'salt'))), 1, 6)) correct_hex,
    CAST(CONCAT('0x', SUBSTR(TO_HEX(MD5(CONCAT(user_id, 'salt'))), 1, 6)) AS INT64) AS numerator,
    CAST('0xFFFFFF' AS INT64) AS denominator,
    CAST(CONCAT('0x', SUBSTR(TO_HEX(MD5(CONCAT(user_id, 'salt'))), 1, 6)) AS INT64) / 
      CAST('0xFFFFFF' AS INT64) AS consistent_rand
  FROM UNNEST(GENERATE_ARRAY(1, 10000)) user_id
```

1. `key`: Allows us to effectively set a seed using `user_id` and a `salt`
1. `md5_key`: Hash the `key` with `MD5()`
1. `hex_md5_key`: Convert `md5_key` with `TO_HEX()`
1. `substr_hex_md5_key`: Take the first 6 digits of `hex_md5_key`
1. `correct_hex`: Prepend 0x to `correct_hex` since all hex values start with 0x
1. `numerator`: Convert `correct_hex` to an integer because we want to get to a number
1. `denominator`: Convert 0xFFFFFF to an integer because it's the largest 6 digit hex value
1. `consistent_rand`: Divide `numerator` and `denominator` to get a value between 0 and 1

And putting this all together for our problem:

```sql
WITH src AS (
  SELECT
    user_id,
    CAST(FLOOR(9 + 10 *
      CAST(CONCAT('0x', SUBSTR(TO_HEX(MD5(CONCAT(user_id, 'salt'))), 1, 6)) AS INT64) / 
      CAST('0xFFFFFF' AS INT64)
    ) AS INT64) AS hour
  FROM UNNEST(GENERATE_ARRAY(1, 10000)) user_id
)

SELECT 
  hour, 
  ARRAY_AGG(user_id) AS user_ids
FROM src
GROUP BY hour
ORDER BY hour
```

## UDFs

We've achieved the goal, but we can clean this up to save ourselves some copy and paste steps in the future. BigQuery let's us define UDFs (user defined functions):

```sql
CREATE OR REPLACE FUNCTION `project.dataset.CONSISTENT_RAND` (
  key STRING
)
RETURNS FLOAT64 AS (
  CAST(CONCAT('0x', SUBSTR(TO_HEX(MD5(key)), 1, 6)) AS INT64) / 
  CAST('0xFFFFFF' AS INT64)
)
```

Now, it's a more convenient query to write:

```sql
WITH src AS (
  SELECT
    user_id,
    CAST(FLOOR(9 + 10 *
      `project.dataset.CONSISTENT_RAND`(CONCAT(user_id, 'salt'))
    ) AS INT64) AS hour
  FROM UNNEST(GENERATE_ARRAY(1, 10000)) user_id
)

SELECT 
  hour, 
  ARRAY_AGG(user_id) AS user_ids
FROM src
GROUP BY hour
ORDER BY hour
```

# Aside on Inverse Transform Sampling

Something implicit in the above example is the mapping from a uniform random number in [0,1], the result of `RAND()` and `CONSISTENT_RAND()`, to a uniform random number between [9, 19], the hour. It's pretty intuitive that you can take something uniform, space the points out a bit, and it stays uniform, but there's also theory behind it -- [Inverse transform sampling](https://en.wikipedia.org/wiki/Inverse_transform_sampling).

For the uniform to uniform case we have our sampler, $ U \sim Unif(0,1) $ and our hours, $ H \sim Unif(9,19) $. 

The [CDF](https://en.wikipedia.org/wiki/Uniform_distribution_(continuous)#Cumulative_distribution_function) for H is:
$$
F(x) = \frac{x-9}{10}
$$
And we want to solve:
$$
F(F^{-1}(u)) = u \\
\frac{F^{-1}(u)-9}{10} = u \\
F^{-1}(u) = 9 + 10 * u
$$

Check it out! We found $ 9 + 10 * u $ which is exactly what we've be writing in our queries: `9 + 10 * CONSISTENT_RAND(key)`.

Now, what if we switch the problem to something slightly different and instead of spreading out our users evenly throughout the day, we want them to spread out according to an exponential distribution. This way more users fall into earlier hours and fewer into later hours -- maybe the downstream work is easier to do early in the day before people get tired. Also let's forget about the working day hours for this part.

For the uniform to exponential case we have our sampler, $ U \sim Unif(0,1) $ and our hours, $ H \sim Expon(1) $.

The [CDF](https://en.wikipedia.org/wiki/Exponential_distribution#Cumulative_distribution_function) for H is:
$$
F(x) = 1 - e^{-x}
$$
And, like before, we want to solve:
$$
F(F^{-1}(u)) = u \\
1 - e^{-F^{-1}(u)} = u \\
e^{-F^{-1}(u)} = 1 - u \\
-F^{-1}(u) = ln(1 - u) \\
F^{-1}(u) = -ln(1 - u)
$$

In BigQuery we can write `-1 * LN(1 - CONSISTENT_RAND(key))` and our groups will follow the exponential distribution.

# A/B tests

You might be asking, what does this have to do with A/B tests? When creating an A/B test, we often want to:
1. Randomly and consistently place users into test groups
1. Allow new users to get into existing groups as they arrive
1. Make sure those assignments don't change over time

This turns out to be very similar to what we have laid out above and we can show it with a quick, familiar query. Our goal is to split our 10,000 users into two evenly sized groups, `A` and `B`:

```sql
WITH src AS (
  SELECT
    user_id,
    `project.dataset.CONSISTENT_RAND`(CONCAT(user_id, 'ab_test_1')) AS u
  FROM UNNEST(GENERATE_ARRAY(1, 10000)) user_id
)

SELECT 
  CASE
    WHEN u >= 0.5 THEN 'A'
    ELSE 'B'
  END AS test_group, 
  ARRAY_AGG(user_id) AS user_ids,
  COUNT(user_id) AS group_size
FROM src
GROUP BY test_group
ORDER BY test_group
```

If we use the combination of `user_id` and the name of the test as the salt, `ab_test_1`, then we can make sure that a user will always be in the same group for that test. And since our pipeline runs many times during the day, any new users will automatically be placed into a group during each run. Last, if we wanted to change from equal group sizes, then we can simply change the `0.5` split to any number within `[0,1]`.

# Conclusion

In the end, we've created a tidy UDF in BigQuery to help create random and consistent values that we can use for making A/B test groups and when working within idempotent data pipelines.